In [1]:
import random
import Levenshtein

In [2]:
alphabet = [
    'a', 'b', 'c', 'd', 'e', 'f', 'g',
    'h', 'i', 'j', 'k', 'l', 'm', 'n',
    'o', 'p', 'q', 'r', 's', 't', 'u',
    'v', 'w', 'x', 'y', 'z', ' '
]

In [3]:
def random_string(l, Sigma):
    string = ""
    for i in range(l):
        value = random.choice(Sigma)
        string += value
    return string

In [4]:
def initialize_population(N, l, Sigma):
    return [random_string(l, Sigma) for x in range(0, N)]

In [5]:
def fraction_correct_characters(string, target):
    assert len(string) == len(target)
    
    correct = 0
    
    for i in range(0, len(string)):
        if string[i] == target[i]:
            correct += 1 
    
    return correct / len(target)

In [6]:
def levenshtein_distance(string, target):
    assert len(string) == len(target)
    
    return Levenshtein.distance(string, target)

In [7]:
def calculate_fitness(string, target, fit_fn):
    if fit_fn == 'frac':
        return fraction_correct_characters(string, target)
    elif fit_fn == 'lev':
        return levenshtein_distance(string, target)
    else:
        return fraction_correct_characters(string, target)

In [8]:
def select_parent(pop, pop_fit, K, fit_fn):
    sel_pool = random.sample(list(zip(pop_fit, range(0, len(pop_fit)))), K)
    
    ind = None
    if fit_fn == 'frac':
        _, ind = max(sel_pool)
    elif fit_fn == 'lev':
        _, ind = min(sel_pool)
    
    return pop[ind]

In [9]:
def cross_over(str1, str2):
    assert len(str1) == len(str2)
    
    ind_to_split = random.randint(0, len(str1) - 1)
    
    return str1[:ind_to_split] + str2[ind_to_split:], str2[:ind_to_split] + str1[ind_to_split:]

In [10]:
def mutate(string, mu, Sigma):
    x = ""
    for i in range(0, len(string)):
        random_number = random.random()
        if mu > random_number:
            x += random_string(1, Sigma)
        else:
            x += string[i]
    
    return x

In [11]:
def step(pop, target, K, mu, Sigma, fit_fn):
    next_pop = []
    best_fit = None
    
    # calculate fitness of individuals in current population
    pop_fit = [calculate_fitness(x, target, fit_fn) for x in pop]
    avg_fit = sum(pop_fit) / len(pop_fit)
    
    if fit_fn == 'frac':
        best_fit = max(pop_fit)
    elif fit_fn == 'lev':
        best_fit = min(pop_fit)
    
    # create next population
    while(len(next_pop) != len(pop)):
        parent1 = select_parent(pop, pop_fit, K, fit_fn)
        parent2 = select_parent(pop, pop_fit, K, fit_fn)
        
        child1, child2 = cross_over(parent1, parent2)
        next_pop.append(mutate(child1, mu, Sigma))
        next_pop.append(mutate(child2, mu, Sigma))
    
    return next_pop, avg_fit, best_fit

In [12]:
def GA(target, K, mu, Sigma, fit_fn = 'frac'):
    l = len(target)
    p_c = 1.0
    N = 1000
    iteration = 0
    
    pop = initialize_population(N, l, Sigma)
    
    while(not target in pop):
        pop, avg_fit, best_fit = step(pop, target, K, mu, Sigma, fit_fn)
        iteration += 1
        print("Iter: {}, avg fit: {}, best fit: {}".format(iteration, avg_fit, best_fit))
        
    return iteration

In [13]:
GA("hello group one", 2, 0.01, alphabet)

Iter: 1, avg fit: 0.03793333333333344, best fit: 0.26666666666666666
Iter: 2, avg fit: 0.062133333333333894, best fit: 0.3333333333333333
Iter: 3, avg fit: 0.09400000000000058, best fit: 0.3333333333333333
Iter: 4, avg fit: 0.12673333333333436, best fit: 0.4
Iter: 5, avg fit: 0.162866666666667, best fit: 0.4
Iter: 6, avg fit: 0.20119999999999988, best fit: 0.4666666666666667
Iter: 7, avg fit: 0.2418666666666675, best fit: 0.5333333333333333
Iter: 8, avg fit: 0.2848000000000003, best fit: 0.6
Iter: 9, avg fit: 0.3325333333333313, best fit: 0.6666666666666666
Iter: 10, avg fit: 0.3741999999999967, best fit: 0.6666666666666666
Iter: 11, avg fit: 0.41866666666666275, best fit: 0.6666666666666666
Iter: 12, avg fit: 0.4614666666666634, best fit: 0.8
Iter: 13, avg fit: 0.5086000000000004, best fit: 0.8
Iter: 14, avg fit: 0.5585333333333357, best fit: 0.8666666666666667
Iter: 15, avg fit: 0.6078666666666707, best fit: 0.8666666666666667
Iter: 16, avg fit: 0.6544000000000031, best fit: 0.933333

16